Dependencies for regular expression matching, csv generation, making web requests and parsing web responses.

In [ ]:
import re
import csv
import requests
from bs4 import BeautifulSoup

Start by getting the valid disclaimer cookie and storing it in a session for future requests.

In [ ]:
session = requests.Session()
accept_disclaimer_url = 'http://icrimewatch.net/session_mgr.php?accepted_license=55753'
disclaimer_page = session.get(accept_disclaimer_url)

Prepare the input and output files.
Input file: newline delimited list of links to detailed offender webpages.
Output file: Four column csv to hold the parsed results.

In [ ]:
in_file = open('offender_urls.txt', 'r')
links = [line.rstrip('\n') for line in in_file]

out_file = open("offenders.csv", "wb+")
c = csv.writer(out_file)
c.writerow(['name','address', 'status', 'link'])

The parsed name, address, and status fields will need to be cleaned up a bit. So prepare some regex to remove wrapping html tags.

In [ ]:
name_regex = re.compile(r'<span class="nameTitle">(.*?)<\/span>')
address_regex = re.compile(r'<td bgcolor="#EEEEEE" colspan="2">(.*?)\s*?.*?<i>(.*?)<\/i>.*?<\/td>')

Loop through each detailed offender url and try to parse the name, address, and status from reponse html. Then compile the data into a csv.

In [ ]:
for link in links:
    page = session.get(link)
    soup = BeautifulSoup(page.text, 'html.parser')

    try:
        raw_name = soup.find_all(class_='nameTitle')[1]
        name = re.search(name_regex, str(raw_name)).group(1).strip()
    except:
        name = "ERROR"

    try:
        raw_address = soup.find(string='Address').find_parent('tr').find_next_sibling().td
        address_parse = re.search(address_regex, str(raw_address))
        address = address_parse.group(1).replace("<br>","").strip()
        status = address_parse.group(2).strip()
    except:
        address = "ERROR"
        status = "ERROR"

    c.writerow([name, address, status, link])

    print name
    print address
    print status
    print link
    print "---------------------------------------"

Close the input and output files. There should now be a csv with 2224 rows of offender data.

In [ ]:
in_file.close()
out_file.close()